In [88]:
from bs4 import BeautifulSoup
import requests
import re
import requests
import pandas as pd

def search_company_page(company):
    company_page = ""
    #Given a company, find its webpage URL through the research system of Reuter

    #This URL return the search page for the given company
    search_url = f'https://www.reuters.com/search/news?sortBy=&dateRange=&blob={company}'

    #Creating url request and bs4 object
    req=requests.get(search_url)

    soup = BeautifulSoup(req.content, "html.parser")
    company_page = soup.find("div",class_="search-stock-ticker").findNext()["href"]

    #we return the final url of the webpage of the company
    return re.sub(r'overview','financial-highlights',"https://www.reuters.com"+company_page)

def get_data(company):
    url=search_company_page(company)
    req=requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")

    ##Get stock price from company page
    try:
        price_L=(soup.
               find("div",class_="sectionQuote nasdaqChange").
               findAll("span"))
        
        price = float(price_L[1].text.strip())
        currency=price_L[2].text.strip()
        
        
    except:
        price = "?"
        currency= "?"

    ##Get %change from company page
    try:
        change = float(re.sub(r'^[(]|[%)]+$',"",
                          soup.
                          find("div",class_="sectionQuote priceChange").
                          findAll("span")[4].
                          contents[0].
                          strip()))/100
        if change == "--":
            change = 0
            
    except:
        change = 0
        
        
    try:
    ##Get Shares owned by institutionnal holders
        shared_owned, =(float(re.sub(r'^[(]|[%)]+$',"",elmt.parent.parent.find(class_="data").contents[0]))
                        for elmt in soup.findAll("strong") 
                        if "Shares Owned" in elmt.contents[0])
    except:
        shared_owned = ""

    
    ##Get divident yield for company/industry/sector
    try:
        yield_list, *rest = (elmt.parent.findAll(class_="data")
                                   for elmt in soup.findAll("td") 
                                   if "Dividend Yield" in elmt.get_text())
    except:
        pass
    
    try:
        company_divident_yield = float(yield_list[0].text)
    except:
        company_divident_yield = "?"
        
    try:    
        industry_divident_yield = float(yield_list[1].text)
    except:
        industry_divident_yield = "?"

    try:    
        sector_divident_yield = float(yield_list[2].text)
    except:
        sector_divident_yield = "?"
    
    try:
        sell_year_ending_list, *rest = (elmt.parent.findAll(class_="data")
                                       for elmt in soup.findAll("td") 
                                       if "Quarter Ending" in elmt.get_text())
        try:
            sell_year_ending_nr = int(sell_year_ending_list[0].text)
        except:
            sell_year_ending_nr = "?"

        try:
            sell_year_ending_mean = float(re.sub(r"[,]","",(sell_year_ending_list[1].text)))
        except:
            sell_year_ending_mean = "?"

        try:
            sell_year_ending_high = float(re.sub(r"[,]","",(sell_year_ending_list[2].text)))
        except:
            sell_year_ending_high = "?"

        try:
            sell_year_ending_low = float(re.sub(r"[,]","",(sell_year_ending_list[3].text)))
        except:
            sell_year_ending_low = "?"

    except:
        sell_year_ending_nr = "?"
        sell_year_ending_mean = "?"
        sell_year_ending_high = "?"
        sell_year_ending_low = "?"   
        
        
    try:
        market = soup.find("span",class_="nasdaqChangeHeader").text.split("on ")[1]
    except:
            marker = "?"
    


    return {"Company":company.upper(),
            "Marker":market,
            "Date":pd.Timestamp.today(tz="UTC").date(),
            "Price":price,
            "Currency":currency,
            "Change":change,
            "Shared_owned":shared_owned,
            "Company_divident_yield":company_divident_yield,
            "Industry_divident_yield":industry_divident_yield,
            "Sell_year_ending_nr":sell_year_ending_nr,
            "Sell_year_ending_mean":sell_year_ending_mean,
            "Sell_year_ending_high":sell_year_ending_high,
            "Sell_year_ending_low":sell_year_ending_low}
        

In [101]:
urlCAC = "https://www.boursier.com/indices/composition/cac-40-FR0003500008,FR.html?tri=dcapi"
reqCAC=requests.get(urlCAC)
soupCAC = BeautifulSoup(reqCAC.content, "html.parser")
CAC40 = [elmt["title"] if "(" not in elmt["title"] else elmt["title"].split("(")[0] for elmt in soupCAC.findAll("a", class_="name--wrapper")]

In [104]:
reuter_L = []
company_list = ["Airbus","Danone","LVMH","total","edf","michelin","carrefour",'valeo','renault','peugeot','technipfmc']
for company in CAC40:
    try:
        reuter_L.append(get_data(re.sub(r" GROUP","",company)))
    except:
        print("Fail "+company)

reuter_df = pd.DataFrame(reuter_L,columns=reuter_L[0].keys())

Fail PSA GROUPE 
Fail ACCORHOTELS


In [105]:
reuter_df.sort_values("Change",ascending=False)

,Company,Marker,Date,Price,Currency,Change,Shared_owned,Company_divident_yield,Industry_divident_yield,Sell_year_ending_nr,Sell_year_ending_mean,Sell_year_ending_high,Sell_year_ending_low
19,ENGIE,Paris Stock Exchange,2018-10-11,12.18,EUR,0.0000,31.26,5.69,3.05,1,20901,20901,20901
1,LVMH,Milan Stock Exchange,2018-10-11,265.05,EUR,0.0000,20.57,1.92,1.70,3,13667.7,13769,13575
21,VIVENDI,Paris Stock Exchange,2018-10-11,21.5,EUR,0.0000,46.57,2.08,0.97,1,0.3,0.3,0.3
22,ARCELORMITTAL,Amsterdam Stock Exchange,2018-10-11,24.76,EUR,0.0000,25.91,0.33,2.60,4,18738.2,19892,17145.3
23,UNIBAIL-WFD,Amsterdam Stock Exchange,2018-10-11,163.4,EUR,0.0000,38.24,4.96,3.86,?,?,?,?
24,RENAULT,Paris Stock Exchange,2018-10-11,71.12,EUR,0.0000,29.52,3.66,2.28,?,?,?,?
25,SAINT-GOBAIN,Paris Stock Exchange,2018-10-11,34.01,EUR,0.0000,38.92,2.56,1.70,1,10551,10551,10551
26,MICHELIN,Paris Stock Exchange,2018-10-11,99.26,EUR,0.0000,39.29,2.67,1.97,?,?,?,?
27,CAPGEMINI,Paris Stock Exchange,2018-10-11,104.55,EUR,0.0000,45.38,1.56,1.35,?,?,?,?
28,LEGRAND,Paris Stock Exchange,2018-10-11,58.46,EUR,0.0000,41.36,1.94,0.96,5,1554.17,1642,1487
